In [1]:
# # -*- coding: utf-8 -*-
# from ckiptagger import data_utils # first use ckip, need to download data file
# data_utils.download_data_gdown("./")
from ckiptagger import WS, POS, NER, construct_dictionary

ws = WS("./ckip_data") # word segment
# pos = POS("./ckip_data") 
# ner = NER("./ckip_data") # entity recognition

import os
import csv
import pandas as pd
import numpy as np
import time
import math
import re
import demoji
from collections import Counter # count most common word
# demoji.download_codes()


#====== import jieba dict =====
import jieba
jieba.set_dictionary('dataset/dic_trad/dict.txt.big')
jieba.load_userdict('dataset/dic_trad/self_define_dict.txt')


#====== selenium activate esc key ======
from selenium.webdriver import ActionChains # activate esc key
from selenium.webdriver.common.keys import Keys # activate esc key

#====== selenium cathc NoSuchElementException ======
from selenium.common.exceptions import NoSuchElementException   
def check_exists_by_xpath(xpath):
    try:
        driver.find_element_by_xpath(xpath)
    except NoSuchElementException:
        return False
    return True

#====== evaluation metrics ======
from sklearn.metrics import mean_squared_error #均方誤差
from sklearn.metrics import mean_absolute_error as mae #平方絕對誤差
from sklearn.metrics import r2_score

#====== MSE ======
def MSE(y, y_pred):
 
    mse = np.mean((y - y_pred)**2)
   
    return mse

/home/joung/anaconda3/envs/mlenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/joung/anaconda3/envs/mlenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/joung/anaconda3/envs/mlenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/joung/anaconda3/envs/mlenv/lib/python3.6/site-


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



Building prefix dict from /home/alvin/essay/dataset/dic_trad/dict.txt.big ...
Loading model from cache /tmp/jieba.ucbd952ca8a83da26cb61ea55c46a2e3b.cache
Loading model cost 0.705 seconds.
Prefix dict has been built successfully.


In [274]:
def load_new_jeiba_word():
    jieba.load_userdict('dataset/dic_trad/self_define_dict.txt') # load self-define dict

In [269]:
#====== get antusd senti dict ======
colNames = ['Word','Score','Pos','Neu','Neg','Non','Not']
antusd = pd.read_csv('dataset/ANTUSD_traditional/opinion_word.csv', names = colNames) 
word_senti_score = dict(zip(antusd['Word'],antusd['Score'])) # find score via word, 27221 words

print(antusd)
# print(word_senti_score)

            Word     Score  Pos  Neu  Neg  Non  Not
0          一下子爆發 -0.388746    0    0    1    0    0
1      一下子爆發的一連串 -0.208389    0    0    1    0    0
2           一夕成名  0.000000    1    0    0    0    0
3           一大打擊 -0.405986    0    0    1    0    0
4            一大步  0.048718    1    0    0    0    0
...          ...       ...  ...  ...  ...  ...  ...
28794         鐐銬  0.000000    0    0    1    3    0
28795         礭實  0.381137    1    0    0    0    0
28796         飆出  0.019193    1    0    0    0    0
28797         攢動  0.000000    0    1    1    2    0
28798         躥起  0.000000    0    1    0    4    0

[28799 rows x 7 columns]


In [272]:
len(antusd)
# antusd['Word']

0            一下子爆發
1        一下子爆發的一連串
2             一夕成名
3             一大打擊
4              一大步
           ...    
28794           鐐銬
28795           礭實
28796           飆出
28797           攢動
28798           躥起
Name: Word, Length: 28799, dtype: object

In [4]:
#====== emoji_sentiment_dict ======
emoji_sentiment = pd.read_csv('dataset/dic_trad/Emoji_Sentiment.csv') # read Emoji_Sentiment.csv
emoji_sentiment.drop(columns=['Unnamed: 0'], inplace = True)
emoji_sentiment_dict = dict(zip(emoji_sentiment['Emoji'],emoji_sentiment['Sentiment score'])) # key,value

In [5]:
def get_selenium_param(filename):
    paramList = []
    with open("dataset/ig_article/"+filename, "r") as f:
        for line in f:
            single_store_param = line.split(',')
            single_store_param[2] = single_store_param[2].replace('\n','')
            paramList.append(single_store_param)

    return paramList

In [6]:
#====== data preprocess - related function ======
def get_store_names(food_kind):
    store_names = [food_kind]
    # read ranked store names
    with open("dataset/ig_article/store_article/"+food_kind+"/ranked_store_names", "r") as f:    
        for line in f:
            store_names.append(line.strip())
        
    return store_names
    
def get_remove_word():
    word_list = []
    with open("dataset/dic_trad/removed_word.txt", "r") as f:    
        for line in f:
            word_list.append(line.strip())
        
    return word_list

In [7]:
#====== extend senti. dict ======
def self_extend_senti_dict(word_senti_score):
    with open("dataset/dic_trad/extend_senti_score_dict.txt", "r") as f:
        for line in f:
            senti_word = line.split()[0]
            senti_score = float(line.split()[1])
            word_senti_score[senti_word] = senti_score


#====== extend senti. dict ======
def self_extend_senti_dict_hotpot(word_senti_score):
    with open("dataset/dic_trad/extend_senti_score_dict_hotpot.txt", "r") as f:
        for line in f:
            senti_word = line.split()[0]
            senti_score = float(line.split()[1])
            word_senti_score[senti_word] = senti_score    
        
#====== add emoji senti score ======
def add_emoji_senti_score(word_senti_score,emoji_sentiment_dict):
    word_senti_score.update(emoji_sentiment_dict)
#     return word_senti_score


self_extend_senti_dict(word_senti_score)
self_extend_senti_dict_hotpot(word_senti_score)
add_emoji_senti_score(word_senti_score, emoji_sentiment_dict)

In [8]:
emoji_sentiment_dict

{'😂': 0.221,
 '❤': 0.746,
 '♥': 0.657,
 '😍': 0.6779999999999999,
 '😭': -0.09300000000000001,
 '😘': 0.701,
 '😊': 0.644,
 '👌': 0.563,
 '💕': 0.632,
 '👏': 0.52,
 '😁': 0.449,
 '☺': 0.657,
 '♡': 0.669,
 '👍': 0.521,
 '😩': -0.368,
 '🙏': 0.41700000000000004,
 '✌': 0.46299999999999997,
 '😏': 0.332,
 '😉': 0.46299999999999997,
 '🙌': 0.5589999999999999,
 '🙈': 0.43200000000000005,
 '💪': 0.555,
 '😄': 0.42100000000000004,
 '😒': -0.374,
 '💃': 0.7340000000000001,
 '💖': 0.7120000000000001,
 '😃': 0.557,
 '😔': -0.146,
 '😱': 0.19,
 '🎉': 0.738,
 '😜': 0.455,
 '☯': 0.001,
 '🌸': 0.65,
 '💜': 0.654,
 '💙': 0.73,
 '✨': 0.35100000000000003,
 '😳': 0.018000000000000002,
 '💗': 0.657,
 '★': 0.28300000000000003,
 '█': -0.032,
 '☀': 0.465,
 '😡': -0.17300000000000001,
 '😎': 0.491,
 '😢': 0.006999999999999999,
 '💋': 0.691,
 '😋': 0.631,
 '🙊': 0.45899999999999996,
 '😴': -0.08,
 '🎶': 0.537,
 '💞': 0.7390000000000001,
 '😌': 0.48200000000000004,
 '🔥': 0.139,
 '💯': 0.12,
 '🔫': -0.19399999999999998,
 '💛': 0.7090000000000001,
 '💁': 0

In [9]:
print(len(word_senti_score),len(emoji_sentiment_dict))

28107 751


In [ ]:
#====== test ckip word segment ======

# text = '傅達仁今將執行安樂死，卻突然爆出自己20年前遭緯來體育台封殺，他不懂自己哪裡得罪到電視台。'
# text = '"鬼金棒，我就跟！就這樣，我又跟我的前衣食父母吃飯！連吃兩天拉麵🍜一個胖的節奏#鬼金棒拉麵#辣正常麻增量#結果真的麻到我頭皮發麻 我到底要不要改機票呢😏"'
# text = '期盼那麼久才吃到，果然不負期望，無論是湯頭還是肉片都是最佳狀態，太享受了'
text = '好久不見！，體力一年不如一年，#20191101，服務態度超優的，海鮮😋,海底撈圈字遊戲送一盤肉,👍👍👍撈撈撈，💋💋💋,排隊排好久 超值得, 我要再來刷二訪 😋， 以愛吃重口味的我來說 麵條的硬度很讚 乾, 想吃很膩害的溏心蛋 這家沾麵真的很優,湯頭也很濃 我不愛沾麵 難波萬台灣美食成為我的愛店之一👍🏻好多人，不知道要排多久,麻的味道超級棒,推推推,湯頭還是很棒，豆芽菜的鑊氣沒了很難過，大概一陣子不會回訪了,good excelent,熱死第一名,可惜還是沒把湯喝完,應該很普通,不是我的最愛。'


ws_results = ws([text])
# pos_results = pos(ws_results)
# ner_results = ner(ws_results, pos_results)

print(ws_results)
# print(pos_results)
# for name in ner_results[0]:# print entity
#     print(name)

In [6]:
# word_pos = dict(zip(ws_results[0],pos_results[0])) # combine word and pos tag

In [26]:
#====== data preprocess - implementation ======
def data_preprocess(articles): 
    # only get string comment & preprocess fans, likes
    all_comments, assigned_index, fans_count, likes  = [],[],[],[]  # assigned_index: denote the original dataframe index of comments
    for i in range(len(articles['text'])):     # remove '\n'
        if type(articles['text'][i]) == str:   # remove non-string
            
            # exclude noise article(ex:染髮)
            temp_word_list = jieba.lcut(articles['text'][i])
            remove_flag = False
            for removed_word in get_remove_word():
                if removed_word in temp_word_list:
                    remove_flag = True
                    break
            if remove_flag:
                continue
                    
            # if contain '\n', replace with ','
            if articles['text'][i].find('\n'): 
                all_comments.append(articles['text'][i].replace('\n','，'))
            else:
                all_comments.append(articles['text'][i])
            assigned_index.append(i)
            
#             # get assigned fans_count
#             if type(articles['fans_count'][i]) == np.float64 and math.isnan(articles['fans_count'][i]):
#                 fans_count.append(0)
#             elif type(articles['fans_count'][i]) == np.float64 and np.isnan(articles['fans_count'][i]):
#                 fans_count.append(0)
#             elif str(articles['fans_count'][i]).find('萬') != -1:
#                 wan_index = articles['fans_count'][i].find('萬')
#                 transform_count = (float(articles['fans_count'][i][:wan_index])) * 10000
#                 fans_count.append(transform_count)
#             else:
#                 fans_count.append(int(articles['fans_count'][i]))
            fans_count = []


            # get assigned likes
            temp_like = articles['likes'][i]
            if type(temp_like) == np.float64 or type(temp_like) == np.int64:
                likes.append(temp_like)
            elif type(temp_like) == str:
                if temp_like.isdigit():
                    likes.append(float(temp_like))
                else:
                    likes.append(1)
            else:
                likes.append(temp_like)
            
        # get article at most 120 post
        if len(all_comments) == 150:
            break
    
    return all_comments, assigned_index, fans_count, likes

In [464]:
#====== cal senti score by jieba ======
jieba.load_userdict('dataset/dic_trad/self_define_dict.txt') # load self-define dict
    
def get_senti_score(text, print_comment):
    '''calculate one sentence's score'''
    if print_comment:
        ws_results = [jieba.lcut(text)] # word segment
        # get sentiword in dict
        total_score = 0
        for word in ws_results[0]:
            if word in word_senti_score:
                score = word_senti_score[word]
                total_score += score 
                print(word,score) # print each senti word's senti score
        print('sent_score:',total_score)

    else:                               # not print one sentence total score
        ws_results = [jieba.lcut(text)] # word segment
        # get sentiword in dict
        total_score = 0
        for word in ws_results[0]:
            if word in word_senti_score:
                score = word_senti_score[word]
                total_score += score 

    
    return total_score

In [ ]:
#====== cal senti score by ckip ======
# def get_senti_score(text):
#     ws_results = ws([text]) # word segment
#     total_score = 0
#     for word in ws_results[0]:
#         if word in word_senti_score:
#             score = word_senti_score[word]
#             total_score += score 
#             print(word,score) # print each senti word's senti score
    
    
#     print('sent_score:',total_score)
#     if total_score >= 1:
#         return 1
#     elif total_score <= -1:
#         return -1
#     else:
#         return total_score

In [461]:
#====== get each comment's score ======
def get_each_comment_score(all_comments, print_comment):
    all_comments_scores = []
    if print_comment:
        for single_comment in all_comments:
            print('\ncomment:',single_comment)
            all_comments_scores.append(get_senti_score(single_comment, print_comment))
    else:
        for single_comment in all_comments:
            all_comments_scores.append(get_senti_score(single_comment, print_comment))
        
    return all_comments_scores

In [15]:
#====== cal store score ======      # not yet consider no comment post's effect (maybe effective) 
def get_store_score(weight, each_comment_score, assigned_index):
    '''store score = comment score * likes(weight) /all likes'''
    weighted_score = 0
    total_weight = 0
    for i in range(len(each_comment_score)):
        if not np.isnan(weight[i]):
            weighted_score += (weight[i] * each_comment_score[i])
            total_weight += weight[i] 
            
    store_score = weighted_score / total_weight

    return store_score


In [18]:
def output_csv(filename, data_list, write_method):

    import csv

    # 開啟輸出的 CSV 檔案
    with open(filename, write_method, newline='') as csvfile:
        writer  = csv.writer(csvfile)   # 建立 CSV 檔寫入器
        for row in data_list: # format: two dimension
            writer.writerow(row)
            

In [ ]:
#====== remove foreign language post ======
# for i,content in zip(articles[articles['text'].notnull()]['text'].index,
#                      articles[articles['text'].notnull()]['text']):
#     if not re.search(r'[\u4e00-\u9fff]+', content) and not demoji.findall(content): # check contain chinese
#         print(i,content)
#         # drop assigned row
#         articles.drop(i, inplace=True) 
        
# # reset dataframe index
# articles.reset_index(drop=True) 

In [19]:
text = '這是一場五感都被放大的饗宴。﻿，﻿，有硬度及Q勁的麵體，🐠很喜歡，搭配辣少麻少的濃厚湯頭，非常過癮！鬼金棒的辣和麻是一流的，入口先感受到辣，隨之而來的麻會讓你感受到舌尖的震顫，建議第一次吃、不是很敢吃麻辣的朋友跟🐠一樣選擇辣少麻少。﻿，﻿，鬼金棒的拉麵，就是這樣強勁有力。﻿，﻿，叉燒肉也和大多數的拉麵店不同，選用大塊的五花肉，乍看之下有點像控肉（但絕對不是），肥瘦相間、軟嫩好入口，有別於一般薄片叉燒，鬼金棒就是這樣的霸氣！﻿，﻿，最後額外提一下，店裡的冰開水不知道為什麼，特別甘甜好喝啊🤣﻿，﻿，﻿，-·-·-·-·-·-﻿，🔺用餐請遵守店內顧客須知，大家一起當優良好客人！﻿，🔺吃完麵再加白飯100分！但麵的份量本身就蠻大了，要自己斟酌哦！﻿，——————————，﻿，🌟如果喜歡濃厚拉麵，或者吃膩了一般口味的拉麵，來鬼金棒走一遭吧☺️﻿，﻿，——————————，🏠辣麻味噌拉麵 鬼金棒﻿，📍104台北市中山區長安西路19巷2弄35號（近捷運中山站）﻿，🕐平日11:30-14:00/17:00-20:30，假日11:30-20:30﻿，，#辣麻味噌拉麵 #鬼金棒 #鬼金棒カラシビ味噌ラーメン #拉麵 #叉燒肉 #捷運美食 #捷運中山站 #ramen #ramennoodles 當然是想到鬼金棒💕，麻的味道超級棒，讓你一口接一口🤤，肉的份量也很足👍，湯頭非常濃郁‼️，吃#nightmarket #taiwan #taipei #noodles #stinkytofu 🤤#squid #seafood #dumplings #ningxia #nightlife,豪豪ㄘ 嘴唇超麻～～～,敲好ㄘ🌶🌶🌶🌶🍜'

# print(ws([text]))
demoji.findall(text)
for emoji in demoji.findall(text):
    if emoji_sentiment_dict.get(emoji):
        print(emoji,emoji_sentiment_dict[emoji])

🏠 0.5
👍 0.521
📍 0.111
🐠 0.414
🌟 0.327
🍜 0.395
💕 0.632


In [ ]:
jieba.lcut('新娘美甲喝起來更香濃愛吃重口味的我來說嗚嗚嗚嗚可惜吃到後面有點膩今天沾麵日！白湯&海湯沾麵🍜')

In [433]:
#======evaluation: read other rank method ======
def read_other_rank(food_kind):
    rank_dict = {'google':{},'ifoodie':{}}
#     food_kind = ['pasta','hotpot','ramen','beef_noodle','pork_rice']
    with open("dataset/ig_article/store_article/other_rank_method",'r') as file:
        for line in file:
            if line.find('google') != -1:        # not found, return -1
                for food in food_kind:
                    if line.find(food) != -1:     # ex:find google pasta rank
                        str_rank_list = line.strip().split(':')[1].split(',')
                        rank_dict['google'][food] = [int(i) for i in str_rank_list]
                        break
                    
                    
                    
#                 if line.find('pasta') != -1:     # find google pasta rank
#                     str_rank_list = line.strip().split(':')[1].split(',')
#                     rank_dict['google']['pasta'] = [int(i) for i in str_rank_list]
                
#                 elif line.find('hotpot') != -1:   # find google pasta rank
#                     str_rank_list = line.strip().split(':')[1].split(',')
#                     rank_dict['google']['hotpot'] = [int(i) for i in str_rank_list]
                
#                 elif line.find('ramen') != -1:   # find google pasta rank
#                     str_rank_list = line.strip().split(':')[1].split(',')
#                     rank_dict['google']['ramen'] = [int(i) for i in str_rank_list]
                    
            elif line.find('ifoodie') != -1:
#                 if line.find('pasta') != -1:
#                     str_rank_list = line.strip().split(':')[1].split(',')
#                     rank_dict['ifoodie']['pasta'] = [int(i) for i in str_rank_list]
                for food in food_kind:
                    if line.find(food) != -1:     # ex:find google pasta rank
                        str_rank_list = line.strip().split(':')[1].split(',')
                        rank_dict['ifoodie'][food] = [int(i) for i in str_rank_list]
                        break    
                    
                    
        
    return rank_dict
            


In [162]:
#====== choose food kind to get mse ======
def get_evaluation_result(food_kind, rank_dict, new_rating_score, store_names, all_store_post_count):
    print('\n===================== '+ food_kind +' rank =====================')
    store_rank = []
    temp_rating = new_rating_score[:]
    temp_rating.sort(reverse = True)
    cnt = 1
    for name,i in zip(store_names[1:],new_rating_score):
        rank = temp_rating.index(i)+1
        store_rank.append(rank)
        print('第'+str(cnt)+'名:',name,round(i,5),',',all_store_post_count[cnt-1],'post',',',rank)
        cnt += 1


    new_mse = MSE(np.array(range(1,11)),np.array(store_rank))
    print('\n新評分機制排名:',store_rank)
    print('MSE:',new_mse)
    print('r2_score:',round(r2_score(np.array(range(1,11)),np.array(store_rank)),3))
    print('--------------------Google--------------------')
    print('Google_rank: ' + str(rank_dict['google'][food_kind]))
    print('Google_MSE:',MSE(np.array(range(1,11)),np.array(rank_dict['google'][food_kind])))
    print('Google_r2_score:',round(r2_score(np.array(range(1,11)),np.array(rank_dict['google'][food_kind])),3))
    print('--------------------ifoodie--------------------')
    print('ifoodie_rank: ' + str(rank_dict['ifoodie'][food_kind]))
    print('ifoodie_MSE:',MSE(np.array(range(1,11)),np.array(rank_dict['ifoodie'][food_kind])))
    print('ifoodie_r2_score:',round(r2_score(np.array(range(1,11)),np.array(rank_dict['ifoodie'][food_kind])),3))
    return new_mse

# Cal one store rating

In [310]:
# all_comments_scores
likes

[49.0,
 10.0,
 26.0,
 68.0,
 4.0,
 7.0,
 107.0,
 3.0,
 3.0,
 28.0,
 18.0,
 144.0,
 10.0,
 52.0,
 2.0,
 15.0,
 15.0,
 29.0,
 6.0,
 12.0,
 13.0,
 13.0,
 12.0,
 41.0,
 11.0,
 13.0,
 37.0,
 60.0,
 6.0,
 59.0,
 7.0,
 45.0,
 762.0,
 9.0,
 nan,
 22.0,
 143.0,
 6.0,
 31.0,
 3.0,
 8.0,
 15.0,
 37.0,
 20.0,
 59.0,
 18.0,
 9.0,
 5.0,
 116.0,
 5.0,
 10.0,
 18.0,
 56.0,
 92.0,
 39.0,
 17.0,
 9.0,
 nan,
 11.0,
 31.0,
 54.0,
 35.0,
 9.0,
 nan,
 5.0,
 14.0,
 33.0,
 18.0,
 81.0,
 22.0,
 47.0,
 4.0,
 33.0,
 64.0,
 7.0,
 17.0,
 23.0,
 6.0,
 2.0,
 20.0,
 101.0,
 9.0,
 41.0,
 11.0,
 74.0,
 nan,
 14.0,
 55.0,
 nan,
 4.0,
 24.0,
 165.0,
 6.0,
 10.0,
 8.0,
 154.0,
 15.0,
 8.0,
 16.0,
 3.0,
 70.0,
 18.0,
 27.0,
 15.0,
 5.0,
 10.0,
 16.0,
 35.0,
 nan]

In [304]:
for i in likes:
    if np.isnan(i):
        print(i)

nan
nan
nan
nan
nan
nan


In [173]:
def norm_score_1(all_comments_scores):
    '''bigger or less than 1/-1 => 1/-1'''
    norm_all_comments_scores = []
    for one_score in all_comments_scores:
        if one_score >= 1:
            norm_all_comments_scores.append(1)
        elif one_score <= -1:
            norm_all_comments_scores.append(-1)
        else:
            norm_all_comments_scores.append(one_score)

    return norm_all_comments_scores

def norm_score_2(x):
    '''max_min norm to [-1,1]'''
    return [(float(i)-np.mean(x))/(max(x)-min(x)) for i in x]

def max_min_normalization(x):
    '''max_min norm to [0,1]'''
    return [(float(i)-min(x))/float(max(x)-min(x)) for i in x]

In [442]:
#====== cal one store rating ======

#====== update senti_dict & jieba_dict ======
self_extend_senti_dict(word_senti_score)
jieba.load_userdict('dataset/dic_trad/self_define_dict.txt') # load self-define dict


# get articles, followers
# fileName = ''

fileName = 'pasta/article_MINT Pasta 義大利麵' #
# fileName = 'pasta/article_螺絲瑪莉義麵坊' # 168
# fileName = 'pasta/article_村口微光' # 180


# fileName = 'hotpot/article_問鼎麻辣鍋忠孝店' # 147
# fileName = 'hotpot/article_滿堂紅bellavita店_松江店' # 101


# fileName = 'article_山嵐拉麵_公館店_台灣總店' # 85 post
# fileName = 'article_山嵐拉麵' # 33 post (remove)
# fileName = 'article_你回來啦 Okaeri 拉麵'# 96
# fileName = 'article_鬼金棒_中山北路' # 126
# fileName = 'article_麵屋壹の穴 ICHI' # 50 (remove)
# fileName = 'article_麵屋壹の穴 ICHI_麵屋壹の穴ichi-沾麵專門店' # 106
# fileName = 'article_柑橘 Shinn' # 69


articles = pd.read_csv(open('dataset/ig_article/store_article/'+fileName+'.csv')) 

# remove '\n', delete non-string
all_comments, assigned_index, fans_count, likes = data_preprocess(articles)  

# get each comment's score
all_comments_scores = get_each_comment_score(all_comments,True)
print(all_comments_scores)

# normalized all comment's senti_score
all_comments_scores = norm_score_1(all_comments_scores)
# all_comments_scores = norm_score_2(all_comments_scores)
# print(all_comments_scores)

# normalized likes
likes = [1 if np.isnan(i) else i for i in likes]
likes = norm_score_1(likes)


# get store rating
# print('\nstore final score:',get_store_score(fans_count, all_comments_scores, assigned_index)) # weight: fans
store_rating = get_store_score(likes, all_comments_scores, assigned_index)
print('\nstore final score:',store_rating) # weight: likes
print('post count:',len(all_comments_scores))






comment: 這個時間最適合放食物照了😘😘，#mint #學生最愛 #平價美食 ，#米蘭茄汁海鮮義大利麵 💰175元，海鮮份量頗多，義大利麵本人也蠻入味，重點是cp值很高啊啊啊😘，，#台北美食 #台湾 #義大利麵 #pasta #lifestyle #foodlover

comment: 思思念念♥，終於吃到你😋😋

comment: 新北市 綠薄荷義大利麵🍝，，CP值很高，深受學生喜愛。，香辣海鮮墨魚麵NT$180，蘑菇濃湯NT$40，建議是早點到避免越晚越多人。，，#台灣生活 #台灣美食 #台湾旅行 #台湾美食 #台湾グルメ #台湾生活 #吃貨人生 #吃貨 #義大利麵 #パスタ #綠薄荷 #景美 #景美夜市 #taiwan #taiwanfood #pasta #gourmet #mintpasta #mintpasta義大利麵

comment: 文青看起來都很高，很用力吸肚子

comment: 粉紅醬起司雞肉麵 原本有微辣（很微），同桌點微辣的朋友覺得會吃膩，但我覺得剛好！，（我點跟我一樣辣度：小辣🌶️），建議吃之前要先攪拌，，不然辣都在底層～越吃越辣🌶️🌶️🌶️

comment: 綠蓉蛤蠣，#義大利麵 #青醬 #青醬義大利麵 #mintpasta #pasta #oyster #basilpasta #景美美食 #taipei #taiwan

comment: 🍝

comment: Mint竟然多了新餐點薯條，紅黃綠三種醬可以選，很像綠蓋的味道，好吃。，奶油煙燻鮭魚的口味超好吃，但不知道為什麼廚師常常漏掉這個口味的調味，已經是第二次了。，最近照片越發越慢了XD

comment: 感謝兩位朋友請吃飯😋，#酥皮南瓜濃湯 ，#燻鴨義大利麵

comment: 世新必吃的義大利麵，然後我實踐的，·，·，·，#taiwan #taipei #igdaily #daily #dailylfe #flatlay #foodporn #yummy #foodie #igtw #foodstagram #instafood #oishi #foods #foodphotography #foody #taiwanesefood #lunch #spaghetti #pasta #pesto ，#台灣 #台北 #午餐 #義大利麵 #相機食先

comment

# Cal one food kind rating

In [468]:
all_food_kinds = ['hongkong']
#====== update senti_dict & jieba_dict ======
self_extend_senti_dict(word_senti_score) 
jieba.load_userdict('dataset/dic_trad/self_define_dict.txt') # load self-define dict

#====== select food kind to get store names ======
for select_food_kind in all_food_kinds:
    store_names = get_store_names(select_food_kind)
    
    rank_dict = read_other_rank(all_food_kinds)
    
    # cal_all_store_rating
    all_store_rating, all_store_post_count = cal_all_store_rating(store_names, True) # data_preprocess: pick at most post
    # evaluation
    get_evaluation_result(select_food_kind, rank_dict, all_store_rating, store_names, all_store_post_count)


-----------------Store Name--------------------
華嫂冰室

comment: 2020.03.16，差點沒讓人揍爆她的10，改不掉遲到壞習慣的10，遲到又不敢接電話的10，遲到最少半小時起跳的10，期待下次約你會不會打破自己不遲到紀錄🤔，，#欸這不是很久沒見的藍醫生嗎
差點 0.0
沒 -0.1
遲到 -0.15541629999999998
遲到 -0.15541629999999998
不敢 -0.4354764
遲到 -0.15541629999999998
起跳 0.0
期待 0.0
下次 0.1
打破 0.0
自己 0.0
不 -0.05
遲到 -0.15541629999999998
不是 -0.7609348000000001
sent_score: -1.8680764

comment: 特地應邀來捧華嫂的冰室
特地 0.11165215
應邀 0.0330536
sent_score: 0.14470575000000002

comment: 上次去台北吃的港式午餐，也是我的口袋名單，就是華嫂冰室！，生意真的好，人很多，但幸運的不用等太久👍🏻我們點的餐點沒有一個是雷的，好吃🤤但價位有點小貴就是了😅，-，#台北#台北美食 #台北午餐#台北晚餐#台北美食地圖#台北下午茶#台北港式#台北小吃 #港式點心#港式飲茶 #港式料理 #西多士#華嫂冰室 #taipei
很多 0.28742399999999996
幸運 0.27024440000000005
不用 -0.2652945
太久 -0.4
👍 0.25
沒有 -0.0921865
好吃 0.37684395
🤤 0.15
美食 0.16
美食 0.16
小吃 0.0
飲茶 0.0
sent_score: 0.8970313500000001

comment: 終於吃到了，等平日不用排隊的華嫂冰室，油蔥雞翼撈丁大推！，，#華嫂冰室#撈丁好吃
終於吃到 0.3
不用 -0.2652945
排隊 -0.1
好吃 0.37684395
sent_score: 0.31154945

comment: 🥢 怎麼看著又餓了
sent_score: 0

comment: 華嫂、華星、華安，傻傻分不清楚，這真的變成我的愛店了❤❤❤❤
傻傻 0.0
變成 0.2

鹹 -0.1
好吃 0.37684395
加油 0.012536499999999999
不用 -0.2652945
排隊 -0.1
sent_score: -0.07591405000000004

comment: 🦾
sent_score: 0

comment: 貳貳捌跟左予心的約會❤️，妹子點名華嫂冰室❤️，招牌芝士鹹牛肉蛋豬仔包❤️，花生西多士❤️，油蔥雞翼撈丁❤️，什麼時候可以去香港呢😂，左左的頭髮會不會被剪的太短😭，什麼時候才可以燙捲捲浪漫波浪😭，2020/02/28✔️ #市政府 #國父紀念館 ，#國父紀念館美食 #市政府美食 ，#華嫂冰室 #港式 #多士 #豬仔包
鹹 -0.1
可以 0.1551797
可以 0.1551797
浪漫 0.11257049999999999
美食 0.16
美食 0.16
sent_score: 0.6429299

comment: 228連假的第一天，漫步在國父紀念館商圈的街頭上，陽光灑下來，金黃的樹葉灑落著一地，感覺有些浪漫，配上港星最愛的茶餐廳*華嫂冰室*，整個很美好😄，，#sunset #sunnyday #day #228 ，#美食 #food #photography #photo ，#國父紀念館 #台北美食 #東區
感覺 0.02
有些 0.25781350000000003
浪漫 0.11257049999999999
最愛 0.5801090999999999
美好 0.8176861999999999
美食 0.16
美食 0.16
sent_score: 2.1081792999999998

comment: 第一次吃華嫂，不知道是退流行了還是因為冠狀病毒影響，人沒有想像中的多
第一次 0.036000533
不 -0.05
知道 0.529169
流行 0.06810095
還是 0.1
影響 -0.05
沒有 -0.0921865
sent_score: 0.5410839829999998

comment: 有誰要來應徵我當手部麻豆😁😁
sent_score: 0

comment: 🥘華嫂冰室，#招牌菠蘿包，#花生西多士 ，#番茄豬扒雞翼通粉
sent_score: 0

comment: _，久違的小聚♥️，港式飲茶真的好好吃🥰
久違 0.0
飲茶 0.0
好吃 0.3768


comment: #20200118，#茗香園 #早知道就聽妹妹的話，真的大部分都又油又難吃😭😭😭，#不好吃就生氣 #厭世の瑄，休想我再去吃下一次！，#發一篇來警惕自己，#西多士 勉強下肚，港式點心還是要去香港吃！
知道 0.529169
妹妹 0.0
大部分 0.048272567
不 -0.05
好吃 0.37684395
生氣 -0.1180638
厭世 0.0
再去 0.1
警惕 0.0
自己 0.0
勉強 0.0
還是 0.1
sent_score: 0.9862217169999998

comment: 蜜汁叉燒飯，超好吃的耶，微甜的肉汁，難怪每次經過都排 那麼多人～～，#中山區美食✌🏻 #台北中山區美食 ，#taipeifoodie ，Friend recommend !，#茗香園冰室
超好吃 0.6
難怪 0.7017641
經過 0.1523594
美食 0.16
美食 0.16
sent_score: 1.7741234999999997

comment: -，我開著夜車走在離地八百公尺的高架快速道路上，歌曲來到落日飛車的勃根地紅，筆直的夜路上除了我暖黃色車燈外，沒有其他車輛，不管順向逆向；順著頭上的銀河直直地油門踩下，末端的星系不知何時抵達，油燈亮起，我想，是該停下來休息的時候了。，-，當你沈溺在滑蛋星空裡伴隨著牛肉火車，隨處飄蕩，活在美好的世界，好比上了KY，把摩擦力降到最低，單純的質量就可以滑去下個行星；一切的美好都毀滅在吃到飯的時候。該起床了喔！澱粉型胖子。，-，🔮茗香園冰室，🛎12:00～22:00，📍台北市大同區南京西路18巷6號2樓，-，#彌樂憨吉 #MLHJ_TW，-，#故事 #幹話 #美食 #小吃 #文字 #感動 #感性 #理性 #peace #love #food #小菜 #冰室 #滑蛋 #夜車
高架 0.0
飛車 0.0
筆直 0.276906
沒有 -0.0921865
不管 -0.5948276
逆向 -0.225
不知 -0.0412733
抵達 0.075
想 0.005
休息 0.18220355
沈溺 0.0
美好 0.8176861999999999
好比 0.0
降到 -0.0804163
最低 -0.2824016
單純 0.075
可以 0.1551797
行星 0.0
美好 0.817686199

發現 0.11151989999999998
蘿蔔 0.0
不 -0.05
愛吃 0.2
蘿蔔 0.0
蘿蔔 0.0
習慣 0.0830536
第一次 0.036000533
喜歡 0.875
😋 0.25
😋 0.25
😋 0.25
肉丸 0.0
😋 0.25
👍 0.25
好吃 0.37684395
美食 0.16
sent_score: 3.0424179830000004

comment: 台北-港式茶餐廳，⭐️⭐️⭐️⭐️，二訪。每一道都很美味，只是用餐時間要排隊等候，值得花半小時等待，其他分店人比較少，但沒試過味道有沒有差別，如果有人試過再留言告訴大家!!，餐點：XO醬招牌公仔麵/流沙奶皇西多士/滑蛋叉燒飯，環境：⭐️⭐️，餐點：⭐️⭐️⭐️⭐️⭐️，CP值：⭐️⭐️⭐️⭐️，地址：103台北市大同區南京西路18巷6號, 樓之1
二訪 0.2
美味 0.6096725000000001
只是 0.0
時間 0.0550611
排隊 -0.1
等候 0.0
值得 0.23
等待 0.2913396
味道 0.0
差別 0.0
告訴 0.575
sent_score: 1.8610731999999999

comment: 港式平價美食，#中山站 #港式 #吃貨人生 #凍檸茶 #蘿蔔糕 #蝦仁粉腸 #沙茶牛肉公仔蛋湯麵 #椒鹽豬扒飯 #茗香園冰室
美食 0.16
蘿蔔 0.0
sent_score: 0.16

comment: 吃一波～開始放假的心情😂，，#茗香園冰室#中山店#港式茶餐廳#流沙奶皇西多士
開始 0.1162332
sent_score: 0.1162332

comment: #台北美食 #中山  #中山美食 #茗香園冰室  #港式 #西多士 #港炒麵 #好立克 #taipei
美食 0.16
美食 0.16
sent_score: 0.32

comment: 超級好吃😋，這個流量是很正常的。
超級 0.0676179
好吃 0.37684395
😋 0.25
正常 0.49881099999999995
sent_score: 1.1932728499999998

comment: ❤️，週六放假，終於預約到高個心心念念的做臉，謝謝 @qinspa1005 的用心，，臉吃完保養品，就到了被推薦的港式茗香園給肚子吃保養品😋，黯然消魂

哈哈 0.07218
不 -0.05
擅長 0.28196709999999997
懷疑 -0.177831
還好 0.3426862
順利 0.0
sent_score: 2.3764039

comment: 來網美都來拍照的泰式港式茶餐廳 點一桌 我們是有多餓 😋，#復古風餐廳📷 #泰式港式茶餐廳 #Day Day Happy Food #吃吃喝喝人生
😋 0.25
sent_score: 0.25

comment: 20200301，🔺美天餐室，🔸清湯牛腱心河粉/$178，🔹無骨酸奶炸雞/$198，🔸Oreo西多士/$110，🔹美祿/$88，📍台北市中山北路二段84巷6號1樓，⏰12:00～9:30，📞02-25216482，-，寒假最後一天來了中山區的一間蠻特別的餐廳，有泰式又有港式的料理，店內的裝潢充滿港式的氛圍卻又播著泰文歌，餐點的部分還算不錯，但是河粉的味道不甚滿意覺得湯頭可以再酸一點點然後豆芽菜竟然是有點生的，但炸雞的部份份量其實蠻多的可以大家一起點一份來平分，而酸奶醬的部份也不會過於搶戲或太膩，西多士跟oreo也是挺搭，如果喜歡吃甜的人應該也會蠻喜歡的，內餡的部份甚至還有花生的味道，整體下來甜而不膩，美祿的部份則是中規中矩，，#霆子食記 #霆子食台北#臺灣美食#台北美食#中山美食#雙連美食#美天餐室#清湯牛肉腱心河粉#無骨酸奶炸雞#oreo西多士#美祿 #港式美食#泰式美食，#instafood#Taiwan#Taipei#food#instagood#foodie#foodietaiwan#Taipeifoodie#Taiwanfood#like4likes #likeforfollow #likeforlikes #vscofood
牛腱 0.0
最後 -0.0175984
特別 0.31378590000000006
充滿 0.4615803
氛圍 0.0
不錯 0.2
味道 0.0
不 -0.05
滿意 0.4838696
覺得 0.02
可以 0.1551797
一點點 -0.06933386700000001
竟然 -0.012536499999999999
其實 0.38113745
可以 0.1551797
平分 0.2568292
不會 -0.27884699999999996
過於 -0.0396672
喜歡 0.875
應

...


comment: 崔記怎麼可以這麼好吃😋
可以 0.1551797
好吃 0.37684395
😋 0.25
sent_score: 0.78202365

comment: 其實我常常內心小劇場都很多，有時候常常睡不著💊，要克服心魔實則不易😓，昨晚品嚐了小熱門的港式小餐館，Cp值：🌟🌟🌟✨
其實 0.38113745
常常 0.147622
內心 0.0
很多 0.28742399999999996
常常 0.147622
睡不著 -0.092331333
克服 0.0
不易 -0.40401929999999997
熱門 0.3667333
sent_score: 0.8341881170000001

comment: 🅺🅰︎🅾︎🅷🆂🅸🆄🅽🅶 | 高雄 |，#前鎮區 吃痣辦抽獎 抽獎LaPinc明天截止‼️，✔︎蔥油撈公仔麵（乾拌麵-去骨雞腿排） $150，✔︎滑蛋什麼飯（自製叉燒） $150，✔︎黯然銷魂飯 自製叉燒+牧場直送太陽蛋 $160，✔︎冰火菠蘿油 $55，✔︎法蘭西多士 $70，✔︎絲襪奶茶-熱 $60，-，一家港式小餐廳，近期常看到人家打卡才注意到他們，店內擺設有著濃濃的復古味，很有特色，有霓虹燈光很像復古電影氛圍，座位不多建議訂位～，🚫低消100元，攜帶外食需收100元清潔費，-，🍛#黯然銷魂飯，來吃港式絕對不能錯過黯然銷魂飯～😋，叉燒味道不錯還可以，不過有看到別人打卡的照片都是半熟蛋，但我們的蛋有點偏七分熟🧐整體來說味道有點單調，醬汁淋的不多，幾乎像是吃白飯，吃完後會有點小空虛～，-，🍲#滑蛋什麼飯 #叉燒，📍牛肉/自製叉燒/時蔬 三選一，滑蛋配上白飯，加上叉燒，真的推👍🏻是今天吃的餐點中最喜歡的一道～叉燒給的份量十足，而且肉真的又好吃，飯上有滿滿的滑蛋，這根本超級幸福🥰不過有些滑蛋火侯控制的不好，蛋很乾會有點像煎蛋不像滑蛋，-，🍝#蔥油撈公仔麵，📍厚切豬扒跟去骨雞腿排 二選一，嫩嫩的雞腿排有加分，很喜歡～好吃😋搭配蔥油吃很香～但個人覺得公仔麵有點乾，一開始拌的時候好像沒有拌勻，發現店家有提供醬油，想說加一點提味用！但醬油絕對不要加太多！！因為這裡的醬油真的很鹹，一定要控制好量😂😂，-，🥜#法蘭西多士，港式下午茶必備甜點，加入蛋汁油煎的吐司，咬下去能咬到奶油香與糖漿的甜味，還能在吐司中吃到滿滿的花生醬，肯定是花生控的最愛，喜歡吃港式下午茶

好吃 0.37684395
感覺 0.02
好像 0.44268620000000003
美食 0.16
美食 0.16
美食 0.16
推薦 0.019754
sent_score: 1.3392841500000001

comment: 有菠蘿油就滿足了🤤
滿足 0.57707005
🤤 0.15
sent_score: 0.72707005

comment: 黯然～～銷魂飯🤤
黯然 -0.012536499999999999
🤤 0.15
sent_score: 0.1374635

comment: 🍳蔥油煎蛋飯｜📍崔記小餐館，#高雄美食 #能吃就是福
美食 0.16
sent_score: 0.16

comment: 🎈高雄-崔記小餐館，🏠：高雄市前鎮區二聖二路67號，⌚️：照片請往後滑就會看到了，⚠️：星期一公休。低消$100，無服務費。，———————————————————————，蔥油撈公仔麵$150 （厚切豬扒/去骨雞腿排），乾炒河粉$160（嫩醃牛肉/港式叉燒/季節時蔬），流沙西多士$100，絲襪奶茶$60，鹹檸七$65，———————————————————————，今天大概六點45到，7點20分左右點餐。，但這個時間叉燒、菠蘿包、牛奶都沒了，，飯類也已經賣完，聽到我也要泛淚了.... 店內有二樓，但是座位不多。，，我本身沒有很愛粗粗的河粉，，但是撇開我個人喜好，，它的調味滿好的河粉很入味，，牛肉也頗嫩。，覺得它是一碗符合台灣人口味的河粉。，，公仔麵我一直覺得很像統一肉燥麵的味道，，對面的捧油一直說不像😂，，今天甜點選擇了流沙西多士，，裡面其實就是流沙包的奶黃餡，，熱熱吃很好吃，我滿喜歡的。，，本來想點的黃金蛋蓋飯、黯然銷魂飯、，波蘿包還有鮮奶茶都點不到😭，建議想用餐的人要早點去，，不然就會跟我一樣沒吃到～，，#高雄#高雄美食 #前鎮區美食 #港式 #蔡董吃什麼 #崔記小餐館 #西多士 #公仔麵 #kaohsiung #dinner #eat
看到 0.0445837
叉燒 0.0
鹹 -0.1
大概 0.045549900000000004
左右 0.0
時間 0.0550611
叉燒 0.0
沒 -0.1
已經 0.029755599999999997
不多 -0.0930434
沒有 -0.0921865
愛 0.2
撇開 -0.


comment: 陪人追星完當然要好好犒賞一下，#港點吃到飽，#蛋堡
當然 0.33799575
sent_score: 0.33799575

comment: Кто там по мордашке хочет???? Сейчас как дам!!!🥰⭐️⭐️⭐️⭐️#смешныеживотные #кошки #коты #котята #котики #cat#cats#kitty#kittys#pussy
🥰 0.25
sent_score: 0.25

comment: 各位女孩們，豐胸曾經是妳們的夢想嗎？，試過各種奇奇怪怪的豐胸產品都沒有效果嗎？，最後眼神死從此覺得自己不配擁有最好的身材？，，沒關係，涵宇都知道！，我的很多客戶試了很多產品都只是單方刺激乳房漲大，不吃之後卻又消失了！對！妳的荷包就跟著它一起消失了😭，，人體胸部發育是個複雜的機制，，怎麼可能單靠一種營養素就可以大了呢？，，所以知道為什麼吃了這麼多單品大了維持不久的原因了嗎？想知道更多快私我諮詢👍，，#豐胸找涵宇  #提高身體抵抗力順便豐胸，，#豐胸 #瘦身 #減肥 #健身，#墾丁 #墾丁白沙灣  #墾丁南灣 #福隆  #墾丁大街 #香港 #新加坡 #澳門 #馬來西亞 #淡水 #淡水漁人碼頭 #淡水老街 #逢甲夜市 #中華航空 #長榮航空 #台北東區 #西門盯 🔍Line : s012435 🔍Wechat : jackhuwang 🔍WhatsApp: +886 973540674
夢想 0.12361110000000002
沒有 -0.0921865
效果 0.1925984
最後 -0.0175984
覺得 0.02
自己 0.0
不配 -0.30944879999999997
擁有 0.2893414
最好 0.5352846
沒關係 0.0
知道 0.529169
很多 0.28742399999999996
客戶 0.0
很多 0.28742399999999996
只是 0.0
刺激 0.0
消失 0.0
消失 0.0
人體 0.0
發育 0.0669362
複雜 -0.475
可能 0.0
可以 0.1551797
知道 0.529169
維持 0.0
想 0.005
知道 0.529169
👍 0.25
提高 0.0
身體 0.0
瘦身 0.0
sent_scor

皮膚 0.0
很好 0.44268620000000003
非常 0.0788721
親切 0.5618249
無視 -0.3453445
肥胖 -0.29758475
患者 -0.09493889999999999
主角 0.0
sent_score: 0.09296214999999991

comment: 肚子餓了，吃點小東西🤤🤤🤤 乾炒牛河，煎蛋牛肉蒸飯，豬肚花生湯，香蔥油條腸粉，水晶鮮蝦餃，流沙菠蘿包
🤤 0.15
🤤 0.15
🤤 0.15
sent_score: 0.44999999999999996

comment: 感謝親愛的，餵飽飽 ！😘，下次再聚聚囉 ~
感謝 0.4296968
親愛 0.7535361999999999
下次 0.1
聚聚 0.0
sent_score: 1.283233

comment: 到了高雄，免不了要找好吃的解解嘴饞，好朋友力推的口袋名單，位在美麗島捷運站，一走出來就可抵達，交通非常方便，且用餐時段，人潮果真絡繹不絕，高朋滿座，港式點心個個讓人驚艷，味道很合我胃口。，，#高雄美食 #高雄美食推薦 #高雄港式料理 #高雄必吃 #捷運站美食 #美麗島站 #美麗島美食 #高雄捷運美食 #高雄捷運美麗島站 #在地人推薦 ，#久違的聚餐，#高雄小叮噹，#小叮噹迷狂人
免不了 0.0
好吃 0.37684395
位在 0.0
美麗 0.6160439
抵達 0.075
非常 0.0788721
方便 0.1761116
人潮 0.0
絡繹不絕 0.267532475
驚艷 0.2573937
味道 0.0
美食 0.16
美食 0.16
推薦 0.019754
美食 0.16
美麗 0.6160439
美麗 0.6160439
美食 0.16
美食 0.16
美麗 0.6160439
推薦 0.019754
久違 0.0
sent_score: 4.5354374250000005

comment: #181229
sent_score: 0

comment: 照很醜所以放IG...，可是還蠻好吃的
很醜 -0.2
可是 -0.17361849999999998
好吃 0.37684395
sent_score: 0.0032254500000000186

comment: 《港味高師父港式點心專賣店》，店名：港味高

sent_score: 0.3220864000000001

comment: 就是很喜歡把桌子點到滿出來，看到就覺得很幸福🥰，這是第4次吃鴨寮街了，然後第一次吃悅誠 老實說比另外一家好吃很多，也比較乾淨，今天幫妹妹慶生。然後送媽媽生日禮物。，很複雜多橋段哈哈，照片多到放不下。，點了10幾道菜。以為自己是美食評審🤣🤣🤣，#雅婷愛食記
喜歡 0.875
看到 0.0445837
覺得 0.02
幸福 0.45
🥰 0.25
第一次 0.036000533
實說 0.0
好吃 0.37684395
很多 0.28742399999999996
乾淨 0.0
妹妹 0.0
送 0.15
複雜 -0.475
哈哈 0.07218
以為 0.03554165
自己 0.0
美食 0.16
sent_score: 2.282573833

comment: 謝謝大師兄強哥的招待！，鴨寮街真的太美味了！，，#用心堅持品質，#謝謝強哥，#生意總是強強滾 ，#快閃高雄一日
謝謝 0.2974824
招待 0.1350369
美味 0.6096725000000001
用心 0.51575515
堅持 0.0
謝謝 0.2974824
總是 0.05
sent_score: 1.90542935

comment: 190129初五，-，之前到悅誠時注意到的港式餐廳，看起來很可以當家族聚餐的地方( ´▽` )ﾉ，烤櫻桃鴨需要事先預訂，剩下的鴨肉和鴨骨可以各二選一烹調方式，坊間的港式和烤鴨店都不常見。我們剩下的鴨肉選擇用銀絲炒鴨條，鴨骨拿去熬芋頭粥。粥裡加了薑和香菇，那個香氣和軟糯感萬分comfort👍，烤鴨皮香酥脆，不會有負擔的油膩感，和鴨肉的連結處也感受不到什麼chewy的臉頰痠，吃起來很輕鬆舒服😌，-，#臺灣#Taiwan#高雄#Kaohsiung#鴨寮街#高雄美食#烤鴨
注意 0.2954928
可以 0.1551797
預訂 0.0
可以 0.1551797
不 -0.05
常見 0.17924555
選擇 0.0210186
香氣 0.2
👍 0.25
不會 -0.27884699999999996
負擔 0.0
連結 0.21993890000000002
不到 -0.4717674
臉頰 0.0
輕鬆 0.0
舒服 0.326906
美食 0.16
sent_score

# Cal all food kind rating

In [453]:
#====== MSE ======
# input store name: [a,b,c]
# output: [score.....]
def cal_all_store_rating(store_names, print_comment):
    if print_comment:
    
        all_store_rating = []
        all_store_post_count = [] # store info dict.
        for i,name in enumerate(store_names[1:]):
            print('-----------------Store Name--------------------')
            print(name)
            fileName = '/article_' + name
            articles = pd.read_csv(open('dataset/ig_article/store_article/'+store_names[0]+fileName+'.csv')) 

            # remove '\n', delete non-string
            all_comments, assigned_index, fans_count, likes = data_preprocess(articles)  

            # get each comment's score
            all_comments_scores = get_each_comment_score(all_comments,print_comment)

            # normalize senti score
    #         all_comments_scores = norm_score_1(all_comments_scores)
    #         all_comments_scores = norm_score_2(all_comments_scores)
            all_comments_scores = max_min_normalization(all_comments_scores)

            # normalized likes
            likes = [1 if np.isnan(i) else i for i in likes]
            likes = norm_score_1(likes)


            # get store rating
            # print('\nstore final score:',get_store_score(fans_count, all_comments_scores, assigned_index)) # weight: fans
            store_rating = get_store_score(likes, all_comments_scores, assigned_index)

            print('\nstore final score:',store_rating) # weight: likes
            all_store_rating.append(store_rating)
            all_store_post_count.append(len(all_comments))
    else:
        all_store_rating = []
        all_store_post_count = [] # store info dict.
        for i,name in enumerate(store_names[1:]):
    #         print('-----------------------------------')
    #         print(name)
            fileName = '/article_' + name
            articles = pd.read_csv(open('dataset/ig_article/store_article/'+store_names[0]+fileName+'.csv')) 

            # remove '\n', delete non-string
            all_comments, assigned_index, fans_count, likes = data_preprocess(articles)  

            # get each comment's score
            all_comments_scores = get_each_comment_score(all_comments,print_comment)

            # normalize senti score
    #         all_comments_scores = norm_score_1(all_comments_scores)
    #         all_comments_scores = norm_score_2(all_comments_scores)
            all_comments_scores = max_min_normalization(all_comments_scores)

            # normalized likes
            likes = [1 if np.isnan(i) else i for i in likes]
            likes = norm_score_1(likes)


            # get store rating
            # print('\nstore final score:',get_store_score(fans_count, all_comments_scores, assigned_index)) # weight: fans
            store_rating = get_store_score(likes, all_comments_scores, assigned_index)

    #         print('\nstore final score:',store_rating) # weight: likes
            all_store_rating.append(store_rating)
            all_store_post_count.append(len(all_comments))
        
        
    return all_store_rating,all_store_post_count

In [466]:
#===================================== main function =======================================
all_food_kinds = ['hotpot','pasta','beef_noodle','pork_rice','curry','hongkong']

#====== update senti_dict & jieba_dict ======
self_extend_senti_dict(word_senti_score) 
jieba.load_userdict('dataset/dic_trad/self_define_dict.txt') # load self-define dict


#====== select food kind to get store names ======
mse_record = {}

for select_food_kind in all_food_kinds:
    store_names = get_store_names(select_food_kind)

    
    rank_dict = read_other_rank(all_food_kinds)
#     print(rank_dict)
    
    # cal_all_store_rating
    all_store_rating, all_store_post_count = cal_all_store_rating(store_names,False) # data_preprocess: pick at most post
    # evaluation matrix
    new_mse = get_evaluation_result(select_food_kind, rank_dict, all_store_rating, store_names, all_store_post_count)
    mse_record[select_food_kind] = new_mse

print('\n'+str(mse_record))


===================== hotpot rank =====================
第1名: 辛殿麻辣鍋 0.25878 , 150 post , 3
第2名: 海底撈火鍋 0.24429 , 150 post , 4
第3名: 老四川巴蜀麻辣燙-南京店 0.29101 , 150 post , 1
第4名: 這一鍋皇室秘藏鍋物 0.22887 , 114 post , 6
第5名: 詹記麻辣火鍋 敦南店 0.24254 , 150 post , 5
第6名: 新馬辣武昌店_新馬辣經典馬辣鍋 0.17124 , 150 post , 8
第7名: 青花驕 麻辣鍋 0.05338 , 134 post , 10
第8名: 馬辣頂級麻辣鴛鴦火鍋 漢口店 0.26408 , 134 post , 2
第9名: 問鼎麻辣鍋忠孝店 0.09509 , 145 post , 9
第10名: 滿堂紅bellavita店_松江店 0.19545 , 101 post , 7

新評分機制排名: [3, 4, 1, 6, 5, 8, 10, 2, 9, 7]
MSE: 7.4
r2_score: 0.103
--------------------Google--------------------
Google_rank: [9, 2, 5, 4, 3, 7, 1, 8, 6, 10]
Google_MSE: 11.8
Google_r2_score: -0.43
--------------------ifoodie--------------------
ifoodie_rank: [6, 3, 5, 4, 2, 10, 1, 9, 7, 8]
ifoodie_MSE: 10.0
ifoodie_r2_score: -0.212

===================== pasta rank =====================
第1名: JAI 宅 0.23696 , 150 post , 6
第2名: MINT Pasta 義大利麵 0.25844 , 109 post , 4
第3名: HUN 混 0.24449 , 150 post , 5
第4名: La PASTA 義大利麵屋 0.18875 , 103 post , 8
第5

In [43]:
mae(np.array(range(1,11)),np.array([9,2,5,4,3,7,1,8,6,10]))

# output ramen rating 
# output_csv('ramen_rating.csv', [[fileName, store_rating]], 'a+')

2.2

# Scrape url via selenium

In [155]:
store_selenium_param = get_selenium_param('selenium_url')
store_names = []
store_page_xpath = []
first_page_xpath = []

for line in store_selenium_param:
    store_names.append(line[0])
    store_page_xpath.append(line[1])
    first_page_xpath.append(line[2])

    
print(first_page_xpath)
# store_names = [store_selenium_param[0][0]]
# store_page_xpath = [store_selenium_param[0][1]]
# first_page_xpath = [store_selenium_param[0][2]]
# get_page_count = [4000]
# print(first_page_xpath)

['href="/p/B_PgFUaDk2y/"', 'href="/p/B_RlyLqA0z8/"', 'href="/p/B_SHtKjJof5/"']


In [156]:
#===== scrape url from first post ======
startTime = time.time()

#====== use selenium to crawl store url ======
from selenium import webdriver

ig_home = "https://instagram.com"
driver = webdriver.Chrome(executable_path="chromedriver_linux64/chromedriver") # Use Chrome
driver.get(ig_home) # open ig home page
time.sleep(2)

# login
driver.find_element_by_xpath('//input[@name="username"]').send_keys('bestronger10142008')
driver.find_element_by_xpath('//input[@name="password"]').send_keys('igcoke1231')
time.sleep(2)
driver.find_element_by_xpath('//button[@type="submit"]').click()
time.sleep(5)

# iterate to get each store url
for store_index in range(len(store_names)):
    # input store name
    driver.find_element_by_xpath('//input[@type="text"]').send_keys(store_names[store_index]) 
    time.sleep(5)
    
    # driver.find_element_by_xpath('//a[@href="/explore/tags/雞吉君拉麵/"]').click() # click store page
    driver.find_element_by_xpath('//a[@' + store_page_xpath[store_index] + ']').click() # click store page
    time.sleep(5)
    
    # driver.find_element_by_xpath('//a[@href="/p/B-CEZjVl8GC/"]').click() # click first page link
    driver.find_element_by_xpath('//a[@' + first_page_xpath[store_index] + ']').click() # click first page link
    time.sleep(3)

    # get url of current page
    url_list, post_date = [], []
    url_list.append([driver.current_url]) # get first page url
    time.sleep(1)
    post_date.append([driver.find_element_by_xpath('//time[@class="_1o9PC Nzb55"]').text])
    time.sleep(3)

    # get 500 page url
#     flag = 0
#     for i in range(get_page_count[store_index]):
    for i in range(8000):
        if check_exists_by_xpath('//a[@tabindex="0"][2]'):
            driver.find_element_by_xpath('//a[@tabindex="0"][2]').click()
        else:
            break

        time.sleep(1)
        url_list.append([driver.current_url])
        time.sleep(5)
        post_date.append([driver.find_element_by_xpath('//time[@class="_1o9PC Nzb55"]').text])
        stopDate = post_date[-1][0]
#         print(post_date[-1][0],type(post_date[-1][0]))
        
        # stop date = 2019.6.1
        if stopDate[:stopDate.find('年')] == '2019' and stopDate[stopDate.find('年')+1:stopDate.find('月')] == '5':
            break
            
    ActionChains(driver).send_keys(Keys.ESCAPE).perform() # close floating page
    output_csv('dataset/ig_article/store_url/url_' + store_names[store_index], url_list, 'w+')
    output_csv('dataset/ig_article/store_url/date_' + store_names[store_index], post_date, 'w+')
    print(store_names[store_index]+'finished!')
    
driver.close()
stopTime = time.time()
print(stopTime-startTime/60)

HUN 混finished!
MINT Pasta 義大利麵finished!
JAI 宅finished!
1561167483.6183445


In [40]:
#===== scrape url from assigned date ======
startTime = time.time()

#====== use selenium to crawl store url ======
from selenium import webdriver

ig_home = "https://instagram.com"
driver = webdriver.Chrome(executable_path="chromedriver_linux64/chromedriver") # Use Chrome
driver.get(ig_home) # open ig home page
time.sleep(2)

# login
driver.find_element_by_xpath('//input[@name="username"]').send_keys('bestronger10142008')
driver.find_element_by_xpath('//input[@name="password"]').send_keys('igcoke1231')
time.sleep(2)
driver.find_element_by_xpath('//button[@type="submit"]').click()
time.sleep(5)

# iterate to get each store url
for store_index in range(len(store_names)):
    # input store name
    driver.find_element_by_xpath('//input[@type="text"]').send_keys(store_names[store_index]) 
    time.sleep(4)
    
    # driver.find_element_by_xpath('//a[@href="/explore/tags/雞吉君拉麵/"]').click() # click store page
    driver.find_element_by_xpath('//a[@' + store_page_xpath[store_index] + ']').click() # click store page
    time.sleep(5)
    
    # driver.find_element_by_xpath('//a[@href="/p/B-CEZjVl8GC/"]').click() # click first page link
    driver.find_element_by_xpath('//a[@' + first_page_xpath[store_index] + ']').click() # click first page link
    time.sleep(4)

    # get url of current page
    url_list, post_date, output_post_date = [], [], []
    post_date.append([driver.find_element_by_xpath('//time[@class="_1o9PC Nzb55"]').text]) # get first page date

    # get 500 page url
    for i in range(5000):
        
        stopDate = post_date[-1][0]
        
        if stopDate[:stopDate.find('年')] != '2019':
#             print(stopDate)
            
            if check_exists_by_xpath('//a[@tabindex="0"][2]'):
                driver.find_element_by_xpath('//a[@tabindex="0"][2]').click() # click next page
                time.sleep(3)
            else:
                break
            time.sleep(1)
            post_date.append([driver.find_element_by_xpath('//time[@class="_1o9PC Nzb55"]').text])
            continue
            
            
#         time.sleep(1)
        url_list.append([driver.current_url])
        time.sleep(1)
        temp_date = [driver.find_element_by_xpath('//time[@class="_1o9PC Nzb55"]').text]
        post_date.append(temp_date)
        output_post_date.append(temp_date)
        
        if check_exists_by_xpath('//a[@tabindex="0"][2]'):
            driver.find_element_by_xpath('//a[@tabindex="0"][2]').click() # click next page
            time.sleep(4)
        else:
            break

        # stop date = 2019.6.1
        if stopDate[:stopDate.find('年')] == '2019' and stopDate[stopDate.find('年')+1:stopDate.find('月')] == '5':
            break
        
            
    ActionChains(driver).send_keys(Keys.ESCAPE).perform() # close floating page
    output_csv('dataset/ig_article/store_url/url_' + store_names[store_index], url_list[:-1], 'w+')
    output_csv('dataset/ig_article/store_url/date_' + store_names[store_index], output_post_date[:-1], 'w+')
    print(store_names[store_index]+'finished!')
    
driver.close()
stopTime = time.time()
print((stopTime-startTime)/60)

新馬辣武昌店finished!
新馬辣經典馬辣鍋finished!
青花驕麻辣鍋板橋縣民大道店finished!
馬辣頂級麻辣鴛鴦火鍋-中山店finished!
馬辣頂級麻辣鴛鴦火鍋 漢口店finished!
問鼎麻辣鍋忠孝店finished!
問鼎 麻辣鍋 養生鍋 粉絲俱樂部finished!
滿堂紅-松江店finished!
滿堂紅bellavita店finished!
153.04108297030132


In [259]:
# teststr = '2019年8月10日'
teststr = '3月23日'
if teststr[:teststr.find('年')] == '2019' and teststr[teststr.find('年')+1:teststr.find('月')] == '8':
    print('yes')

# Extend antusd

In [346]:
def remove_punctuation(line):
    punctuation = "·*$,~-:!。; ！？｡＂＃＄％＆＇（）＊＋，－/\／：；＜＝＞#＠［＼］＾＿｀｛｜｝～()｟｠｢｣､、〃》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘'‛“”„‟…‧﹏."
    punctuation = [i for i in punctuation] + ['\n','--','☆','⋯','..']
    line = [word for word in line if word not in punctuation and not word.isdigit()]

    return line

In [363]:
#====== word segment ======
'''read every file in google_review'''
all_reviews = []       # include 1~5 stars review
for filename in os.listdir('mac_extended_dict/google_review/'):              
    with open('mac_extended_dict/google_review/'+filename, newline='') as f: # open in readonly mode
        rows = csv.reader(f)
        firstline = True
        for post in rows:           # read each user post
           
            if firstline == True:   # skip fisrt line
                firstline = False
                continue
            tmp_star = int(post[2].strip()[0])
            tmp_review = post[3].strip().replace('(由 Google 提供翻譯)','').replace('(原始評論)','')
            if tmp_review == '':    # skip empty review
                continue
            all_reviews.append([tmp_star,tmp_review]) # extract star & review
            

all_reviews = np.array(all_reviews)



In [375]:
#====== filter new extended antusd words ======
def filter_new_ntusd_word(antusd, all_reviews):
    extended_antusd_word = []
    antusd_list = list(antusd['Word'])
    load_new_jeiba_word()
    new_extend_word = []
    for one_review in all_reviews[:,1]:               # get each review
        one_review = one_review.upper()
        one_review = one_review.replace('.','')
        one_review = one_review.replace(' ','')

        if one_review.find('C/P') != -1:              # found C/P => CP
            one_review = one_review.replace('/','')

        for word in remove_punctuation(jieba.lcut(one_review)): # get each word
            if word not in antusd_list and len(word) > 1:       # filter not in antusd & get rid of single letter
                extended_antusd_word.append(word)

#     print(len(extended_antusd_word))
    return extended_antusd_word

In [394]:
#====== get 5 star and 1 star review  ======
five_star_review, one_star_review, = [],[]
for one_review in all_reviews:
    if one_review[0] == '5':
        five_star_review.append(list(one_review))
    elif one_review[0] == '1':
        one_star_review.append(list(one_review))
        
        
five_star_review = np.array(five_star_review)
one_star_review = np.array(one_star_review)
print(len(five_star_review),len(one_star_review))
        

8732 1712


In [399]:
# get pos words & neg words from 5 star & 1 star review 
pos_words = filter_new_ntusd_word(antusd, five_star_review)
neg_words = filter_new_ntusd_word(antusd, one_star_review)

In [403]:
print('new negative words:',len(neg_words),'\nnew positive words:',len(pos_words))

new negative words: 19218 
new positive words: 70511


In [420]:
# count_pos_words = Counter(pos_words).most_common()
# output_csv('Count_pos_word',count_pos_words,'w') # don't execute
# count_neg_words = Counter(neg_words).most_common()
# output_csv('Count_neg_word',count_neg_words,'w') # don't execute

In [422]:
def get_noted_word(filename):
    noted_word = []
    with open(filename,newline='') as f:
        rows = csv.reader(f)
        for row in rows:
            if len(row) == 3:
                noted_word.append(row)
#                 print(row)
                
    return noted_word

new_antusd_pos_word = get_noted_word('Count_pos_word')
print(len(new_antusd_pos_word))

708


In [426]:
new_antusd_neg_word = get_noted_word('Count_neg_word')
print(len(new_antusd_neg_word))

314


# combine csv

In [317]:
article1 = pd.read_csv('dataset/ig_article/store_article/pasta/article_MIGA KITCHEN PASTA.csv')
article2 = pd.read_csv('dataset/ig_article/store_article/pasta/article_米家廚房.csv')
combined_article = pd.concat([article1, article2], ignore_index=True)
combined_article.to_csv('dataset/ig_article/store_article/pasta/article_MIGA KITCHEN PASTA_米家廚房.csv')

# extend dictionary

In [413]:
# extend dictionary
word_to_weight = {
    "土地公": 1,
    "土地婆": 1,
    "公有": 2,
    "緯來體育台": 1,
    "鬼金棒": 3,
    "海底撈": 5
}
dictionary = construct_dictionary(word_to_weight)
print(dictionary)

[(2, {'公有': 2.0}), (3, {'土地公': 1.0, '土地婆': 1.0, '鬼金棒': 3.0, '海底撈': 5.0}), (5, {'緯來體育台': 1.0})]
